In [3]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ['HF_TOKEN'] = user_secrets.get_secret("HF_TOKEN")
os.environ['OPENAI_API_KEY'] = user_secrets.get_secret("OPENAI_API_KEY")

In [4]:
!pip install transformers==4.40.1 bitsandbytes==0.43.1 accelerate==0.29.3 datasets==2.19.0 tiktoken==0.6.0 -qqq

## 기존 모델 활용 

In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("beomi/Yi-Ko-6B")
model = AutoModelForCausalLM.from_pretrained("beomi/Yi-Ko-6B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.51k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00005.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/2.86G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/643M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [6]:
DDL = f'CREATE TABLE players ( player_id INT PRIMARY KEY AUTO_INCREMENT, username VARCHAR(255) UNIQUE NOT NULL, email VARCHAR(255) UNIQUE NOT NULL, password_hash VARCHAR(255) NOT NULL, date_joined DATETIME NOT NULL, last_login DATETIME );'
Question = f'모든 플레이어 정보를 조회해 줘'
SQL = f''

example = f'''
당신은 sql 생성하는 전문가야 
DDL 활용해 Question 해결하는 
SQL query 생성
추측 과정은 빼고 작성해줘
{DDL} 
{Question} 
{SQL}
'''

In [7]:
example

'\n당신은 sql 생성하는 전문가야 \nDDL 활용해 Question 해결하는 \nSQL query 생성\n추측 과정은 빼고 작성해줘\nCREATE TABLE players ( player_id INT PRIMARY KEY AUTO_INCREMENT, username VARCHAR(255) UNIQUE NOT NULL, email VARCHAR(255) UNIQUE NOT NULL, password_hash VARCHAR(255) NOT NULL, date_joined DATETIME NOT NULL, last_login DATETIME ); \n모든 플레이어 정보를 조회해 줘 \n\n'

In [8]:
## tokenizing
inputs = tokenizer.encode(example, return_tensors='pt')

In [9]:
len(inputs[0])

128

In [10]:
# 모델 예측 수행(생성 글 작성)
outputs = model.generate(inputs,
               max_new_tokens=100,
               pad_token_id=tokenizer.eos_token_id,
               temperature=0.5
              )
outputs

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


tensor([[    1, 59568,   144, 76851, 71613, 33162, 64069, 76858, 64018, 65475,
         76848, 59568,   144, 14499, 59620, 69033, 24983, 65192, 64018, 59568,
           144, 14666,  8975, 64069, 76858,   144, 76992, 77248, 64230, 65028,
         67533, 68254, 63934, 77270,   144, 57739, 30589,  3411,   662,  3875,
         59593,   619, 11710,  6123,  3811, 15914, 34341, 58552, 59593,  1625,
         26406, 12766,    97, 20576,  1008, 23778,  1955, 59605,    79,    82,
            82, 59604,  8568, 59597, 24849,  5928,  8987,    97,  3449,  1008,
         23778,  1955, 59605,    79,    82,    82, 59604,  8568, 59597, 24849,
          5928,  8987,    97,  8355, 59593, 14529,  1008, 23778,  1955, 59605,
            79,    82,    82, 59604,  5928,  8987,    97,  3817, 59593,  9522,
          1559,   723,  1765,  3013, 13862,  5928,  8987,    97,  1426, 59593,
         17817,   723,  1765,  3013, 13862, 58946, 59568,   144, 71386, 67763,
         76816, 68724, 72178, 63934, 69305, 59568,  

In [11]:
# 생성 된 text 출력 
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
result

" \n당신은 sql 생성하는 전문가야 \nDDL 활용해 Question 해결하는 \nSQL query 생성\n추측 과정은 빼고 작성해줘\nCREATE TABLE players ( player_id INT PRIMARY KEY AUTO_INCREMENT, username VARCHAR(255) UNIQUE NOT NULL, email VARCHAR(255) UNIQUE NOT NULL, password_hash VARCHAR(255) NOT NULL, date_joined DATETIME NOT NULL, last_login DATETIME ); \n모든 플레이어 정보를 조회해 줘 \n\nSELECT * FROM players; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = 'jonghoon'; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = 'jonghoon' AND password_hash = '123456'; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = 'jonghoon' AND password_hash = '123456' AND date_"

### 모델을 한번에 동작하게 작성


In [12]:
import torch
from transformers import BitsAndBytesConfig, pipeline

def make_inference_pipeline(model_id):
    # tokenizer 
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    # 버전 호환 문제로 삭제
    # # 양자화(모델 사이즈 축소) 
    # quantization_config = BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_compute_dtype=torch.float16
    # )
    # model 
    model = AutoModelForCausalLM.from_pretrained(model_id, 
                                                 # quantization_config=quantization_config,
                                                 load_in_4bit=True, 
                                                 bnb_4bit_compute_dtype=torch.float16, # 양자화 정의
                                                 device_map='auto'
                                                )
    
    # pipeline : 예측 초기화 설정
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

    return pipe

In [13]:
model_id = f"beomi/Yi-Ko-6B"
hf_pipe = make_inference_pipeline(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [17]:
results = hf_pipe(example, do_sample=False,
       return_full_text=False, max_length=512, truncation=True)
results

[{'generated_text': "SELECT * FROM players; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = '김철수'; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = '김철수' AND password_hash = '12345678'; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = '김철수' AND password_hash = '12345678' AND date_joined = '2019-01-01'; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = '김철수' AND password_hash = '12345678' AND date_joined = '2019-01-01' AND last_login = '2019-01-01'; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = '김철수' AND password_hash = '12345678' AND date_joined = '2019-01-01' AND last_login = '2019-01-01' AND date_joined > '2019-01-01'; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = '김철수' AND password_hash = '12345678' AND date_joined = '2019-01-01' AND last_login = '2019-01-01' AND date_joined > '2019-01-01'; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = '김철수' AND password_hash = '"}]

In [16]:
type(hf_pipe), type(pipeline)

(transformers.pipelines.text_generation.TextGenerationPipeline, function)

In [22]:
results[0].keys()

dict_keys(['generated_text'])

In [24]:
results[0]['generated_text']

"SELECT * FROM players; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = '김철수'; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = '김철수' AND password_hash = '12345678'; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = '김철수' AND password_hash = '12345678' AND date_joined = '2019-01-01'; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = '김철수' AND password_hash = '12345678' AND date_joined = '2019-01-01' AND last_login = '2019-01-01'; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = '김철수' AND password_hash = '12345678' AND date_joined = '2019-01-01' AND last_login = '2019-01-01' AND date_joined > '2019-01-01'; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = '김철수' AND password_hash = '12345678' AND date_joined = '2019-01-01' AND last_login = '2019-01-01' AND date_joined > '2019-01-01'; \n플레이어 정보 조회 \n\nSELECT * FROM players WHERE username = '김철수' AND password_hash = '"